In [1]:
import geopandas as gpd
import pandas as pd

In [3]:
# 1️⃣ Point to your GeoJSONs and target CRS for Lausanne
INPUTS = {
    1800: "src/data/Berney_espaces_verts_macro.geojson",
    1830: "src/data/espaces_verts_macro.geojson",
    2024: "src/data/2024_espaces_verts_macro.geojson",
}
# CH1903+ / LV95 is EPSG:2056, good for Swiss metres
TARGET_CRS = "EPSG:2056"

# 2️⃣ Collect raw per-feature areas
records = []
for year, path in INPUTS.items():
    print(f"Processing {year} → {path}")
    gdf = gpd.read_file(path)
    # ensure the GeoJSON has a CRS; if not, set its original CRS (likely EPSG:4326)
    if gdf.crs is None:
        gdf = gdf.set_crs("EPSG:4326")
    gdf = gdf.to_crs(TARGET_CRS)
    gdf["area_m2"] = gdf.geometry.area
    # expect a `macro_ev` column and maybe `merge_id`
    for _, row in gdf.iterrows():
        records.append({
            "year":       year,
            "merge_id":   row.get("merge_id", ""),
            "class":      row["macro_ev"],
            "area_m2":    row["area_m2"],
        })

# 3️⃣ Write the raw table (one row per polygon)
df = pd.DataFrame(records)
df.to_csv("src/data/areas_raw.csv", index=False)

# 4️⃣ Aggregate by year + class
summary = (
    df
    .groupby(["year", "class"], as_index=False)
    .area_m2
    .sum()
    .rename(columns={"area_m2": "total_area_m2"})
)
summary.to_csv("src/data/areas_by_class.csv", index=False)

print("Wrote src/data/areas_raw.csv and src/data/areas_by_class.csv")

Processing 1800 → src/data/Berney_espaces_verts_macro.geojson


Skipping field own_col_de: unsupported OGR type: 5


Processing 1830 → src/data/espaces_verts_macro.geojson
Processing 2024 → src/data/2024_espaces_verts_macro.geojson
Wrote src/data/areas_raw.csv and src/data/areas_by_class.csv
